In [3]:
import pandas as pd
import numpy as np
from collections import defaultdict
from matplotlib import pyplot as plt

### CREATE AGG MATRIX AND READ DATA

In [4]:
data = pd.read_csv("data.csv", index_col=0) # 1 + 1*4 + 1*4*7 + 1*4*7*2
agg_mat_df = pd.read_csv("agg_mat.csv", index_col=0) # matrix of aggregated data with bottom time series

In [5]:
data

,total,hol,vfr,bus,oth,nsw-hol,vic-hol,qld-hol,sa-hol,wa-hol,...,qld-oth-city,qld-oth-noncity,sa-oth-city,sa-oth-noncity,wa-oth-city,wa-oth-noncity,tas-oth-city,tas-oth-noncity,nt-oth-city,nt-oth-noncity
1998-03-31,84503,45906,26042,9815,2740,17589,10412,9078,3089,3449,...,431,271,244,73,168,37,76,24,35,8
1998-06-30,65312,29347,20676,11823,3466,11027,6025,6310,1935,2454,...,669,170,142,221,170,99,36,61,69,39
1998-09-30,72753,32492,20582,13565,6114,8910,5060,11733,1569,3398,...,270,1164,397,315,380,1166,32,23,150,338
1998-12-31,70880,31813,21613,11478,5976,10658,5481,8109,2270,3561,...,214,535,194,260,410,1139,48,43,172,453
1999-03-31,86893,46793,26947,10027,3126,16152,10958,10047,3023,4287,...,458,557,147,33,162,28,77,60,15,47
1999-06-30,66866,31442,19481,11875,4068,9840,5841,8088,2219,3905,...,364,555,342,129,125,161,70,90,72,63
1999-09-30,72182,34623,20026,11713,5820,9265,5104,12553,1712,3729,...,935,1865,137,109,546,243,26,49,75,182
1999-12-31,68318,31118,20431,10582,6187,10427,4774,9652,1940,2732,...,640,703,400,251,299,568,34,56,145,217
2000-03-31,85651,47030,24284,9734,4603,16340,9972,10077,3282,5131,...,513,374,274,155,365,266,49,122,37,56
2000-06-30,64467,30852,19430,10590,3595,9797,5758,8564,1845,3324,...,298,249,348,84,212,299,33,47,41,45


In [268]:
### create global matrix A
### create list of number of TS in each level
levels = [0, 1,1*4,1*4*7,1*4*7*2]
levels_left = [0, 1, 5, 5+28]
levels_right = [0, 4, 4+28, 4+28+56]
nb_ts_levels = [1, 4, 28, 56]
nb_ts_agg_levels = [1,4,7,2]

def matrix_per_level(global_Matrix, levels_left, levels_right, l):
    return np.array(global_Matrix.iloc[levels_left[l-1]:levels_left[l], levels_right[l-1]:levels_right[l]])

def creat_agg_mat(nb_ts_levels, nb_ts_agg_levels):
    nb_total_ts = sum(nb_ts_levels)
    nb_ts_agg =sum(nb_ts_levels[:len(nb_ts_levels)-1])
    global_Matrix = pd.DataFrame(np.zeros((nb_ts_agg, nb_total_ts-1)))
    
    
    for j in range(0,nb_ts_levels[1]):
        global_Matrix.iloc[0,j] = 1
    i=1
    for k in range(nb_ts_levels[1]):
        for j in range(4+k*7,4+(k+1)*7):
            global_Matrix.iloc[i,j] = 1
            #print(i,j)
        i+=1
    for k in range(nb_ts_levels[2]):
        for j in range(4+28+k*2,4+28+(k+1)*2):
            global_Matrix.iloc[i,j] = 1
            #print(i,j)
        i+=1
            

    return global_Matrix

In [269]:
A = creat_agg_mat(nb_ts_levels, nb_ts_agg_levels)
A

,0,1,2,3,4,5,6,7,8,9,...,78,79,80,81,82,83,84,85,86,87
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [271]:
matrix_per_level(A, levels_left, levels_right, 1).T

array([[1.],
       [1.],
       [1.],
       [1.]])

### MODEL

In [133]:
# Split the data
### pivot data such as index is the name of columns
#data = data.pivot(index='date', columns='symbol', values='close')
pivot_df = data.T

X_train = pivot_df.iloc[:,:17]
#X_val = data.iloc[nb_train:nb_train+nb_val,:]
X_test = pivot_df.iloc[:,18:35]

y_train = pivot_df.iloc[:,17]
#y_val = data.iloc[nb_train+nb_val,:]
y_test = pivot_df.iloc[:,35]

In [117]:
X_test.shape

(89, 17)

In [167]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.layers import Dense, Input, Dropout
# Create a sequential model


inp = Input(shape=(17,))
hid1 = Dense(100, activation='relu')(inp)
h2_w  = Dense(89, activation='linear')(hid1)
out = Dense(1, activation='linear')(h2_w)

mdl = Model(inputs=inp, outputs=out)
mdl.compile(loss='mse', optimizer='adam')



mdl.fit(X_train,y_train,epochs=100, batch_size=10)

#mdl.fit(X_train,Y_train, shuffle=True, validation_data=[X_test, Y_test], epochs=20, batch_size=32)
#p = mdl.predict(X_test)
#np.sqrt(((np.log1p(p) - np.log1p(Y_test))**2)).mean(axis=0)


Epoch 1/100
9/9 [==============================] - 0s 2ms/step - loss: 6215463.0000
Epoch 2/100
9/9 [==============================] - 0s 1ms/step - loss: 2414759.2500
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 7435009.5000
Epoch 4/100
9/9 [==============================] - 0s 2ms/step - loss: 9295457.0000
Epoch 5/100
9/9 [==============================] - 0s 6ms/step - loss: 2202381.7500
Epoch 6/100
9/9 [==============================] - 0s 1ms/step - loss: 4376214.5000
Epoch 7/100
9/9 [==============================] - 0s 2ms/step - loss: 9097378.0000
Epoch 8/100
9/9 [==============================] - 0s 2ms/step - loss: 3230828.2500
Epoch 9/100
9/9 [==============================] - 0s 3ms/step - loss: 131405.7812
Epoch 10/100
9/9 [==============================] - 0s 2ms/step - loss: 168722.4219
Epoch 11/100
9/9 [==============================] - 0s 2ms/step - loss: 81332.3906
Epoch 12/100
9/9 [==============================] - 0s 2ms/step - loss: 140857

In [165]:
weights_hid1_to_h2_w = mdl.layers[2].get_weights()[0]
weights_hid1_to_h2_w.shape

(100, 89)

In [158]:
mdl.summary()

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 17)]              0         
                                                                 
 dense_29 (Dense)            (None, 100)               1800      
                                                                 
 dense_30 (Dense)            (None, 89)                8989      
                                                                 
 dense_31 (Dense)            (None, 1)                 90        
                                                                 
Total params: 10,879
Trainable params: 10,879
Non-trainable params: 0
_________________________________________________________________


In [159]:
y_predict = np.array(mdl.predict(X_test)).reshape(-1)

3/3 [==============================] - 0s 7ms/step


In [160]:
def calculate_mape(actual_values, forecasted_values):
    n = len(actual_values)
    mape = np.mean(np.abs((actual_values - forecasted_values) / actual_values)) * 100
    return mape

In [161]:
calculate_mape(y_test, y_predict)

23.370994112381428

### MODEL FOR HIERARCHICAL TIME SERIES

In [254]:
lengths = [1, 4, 28, 56]
def reshape_weights(weight, lengths):
    start_idx = 0
    result = []
    for length in lengths:
        sub_array = weight[start_idx:start_idx + length]
        result.append(sub_array)
        start_idx += length

    result = np.array(result)
    return result

In [310]:
def loss_up(weight_matrix):
        weights_list = weight_matrix.numpy()
        
        n, m = weights_list.shape
        
        regularization_loss = 0

        for i in range(n):
            w = reshape_weights(weights_list[i,:], lengths)
            for l in range(len(w) - 1):  # Define 'l' in the outer loop
                for j in range(len(w[l])):
                      regularization_loss += w[l][j] - np.dot(matrix_per_level(A, levels_left, levels_right, l + 1)[j], w[l + 1])
                

        return regularization_loss
loss_up(weight_matrix)

C:\Users\hamza\AppData\Local\Temp\ipykernel_19716\1877181359.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.array(result)


-3.991486356842918

In [317]:
keras.backend.sum(keras.backend.square(weight_matrix))

<tf.Tensor: shape=(), dtype=float32, numpy=83.80679>

In [316]:
def custom_loss_with_regularization(reg_weight, weight_matrix):
    def loss(y_true, y_pred):
        mse_loss = keras.losses.mean_squared_error(y_true, y_pred)  # MSE loss
        regularization_loss = reg_weight * keras.backend.sum(keras.backend.square(weight_matrix))  # Regularization term using the weight_matrix
        
        total_loss = mse_loss + regularization_loss
        return total_loss
        print(total_loss)
    return loss

# Create a sequential model
inp = Input(shape=(17,))
hid1 = Dense(100, activation='relu')(inp)
h2_w = Dense(89, activation='linear')(hid1)
out = Dense(1, activation='linear')(h2_w)

mdl = Model(inputs=inp, outputs=out)

# Compile the model with the custom loss function and an optimizer
custom_loss = custom_loss_with_regularization(reg_weight=0.01, weight_matrix=mdl.layers[2].kernel)
optimizer = Adam(learning_rate=0.001)
mdl.compile(loss=custom_loss, optimizer=optimizer)

# Assuming you have X_train and y_train
mdl.fit(X_train, y_train, epochs=100, batch_size=10)

Epoch 1/100


c:\Users\hamza\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


9/9 [==============================] - 0s 20ms/step - loss: 12308410.0000
Epoch 2/100
9/9 [==============================] - 0s 23ms/step - loss: 1598085.8750
Epoch 3/100
9/9 [==============================] - 0s 22ms/step - loss: 1452786.0000
Epoch 4/100
9/9 [==============================] - 0s 18ms/step - loss: 2936768.7500
Epoch 5/100
9/9 [==============================] - 0s 19ms/step - loss: 8665387.0000
Epoch 6/100
9/9 [==============================] - 0s 19ms/step - loss: 37308852.0000
Epoch 7/100
9/9 [==============================] - 0s 17ms/step - loss: 19864154.0000
Epoch 8/100
9/9 [==============================] - 0s 19ms/step - loss: 3101913.2500
Epoch 9/100
9/9 [==============================] - 0s 20ms/step - loss: 25026330.0000
Epoch 10/100
9/9 [==============================] - 0s 18ms/step - loss: 6860693.0000
Epoch 11/100
9/9 [==============================] - 0s 18ms/step - loss: 2466755.2500
Epoch 12/100
9/9 [==============================] - 0s 17ms/step - loss

In [318]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam

# Assuming you have X_train and y_train
# Assuming you have lengths, A, levels_left, and levels_right defined

# Enable eager execution
tf.config.run_functions_eagerly(True)

# Define your custom loss function
def custom_loss_with_regularization(reg_weight, lengths, A, levels_left, levels_right):
    def loss(y_true, y_pred):
        mse_loss = keras.losses.mean_squared_error(y_true, y_pred)  # MSE loss
        
        def loss_up(weight_matrix):
            weights_list = weight_matrix.numpy()
            n, m = weights_list.shape
            regularization_loss = 0

            for i in range(n):
                w = reshape_weights(weights_list[i, :], lengths)
                for l in range(len(w) - 1):
                    for j in range(len(w[l])):
                        regularization_loss += w[l][j] - np.dot(matrix_per_level(A, levels_left, levels_right, l + 1)[j], w[l + 1])
                
            return regularization_loss
        
        custom_reg_loss = loss_up(weight_matrix)
        total_loss = mse_loss + reg_weight * custom_reg_loss
        return total_loss
    
    return loss

# Create a sequential model
inp = Input(shape=(17,))
hid1 = Dense(100, activation='relu')(inp)
h2_w = Dense(89, activation='linear')(hid1)
out = Dense(1, activation='linear')(h2_w)

mdl = Model(inputs=inp, outputs=out)

# Compile the model with the custom loss function and an optimizer
custom_loss = custom_loss_with_regularization(1, lengths, A, levels_left, levels_right)
optimizer = Adam(learning_rate=0.001)
mdl.compile(loss=custom_loss, optimizer=optimizer)

mdl.fit(X_train, y_train, epochs=100, batch_size=10)

Epoch 1/100


c:\Users\hamza\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\hamza\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py:371: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return py_builtins.overload_of(f)(*args)


9/9 [==============================] - 20s 2s/step - loss: 1983681.6250
Epoch 2/100
9/9 [==============================] - 17s 2s/step - loss: 1599286.2500
Epoch 3/100
9/9 [==============================] - 14s 2s/step - loss: 1588319.8750
Epoch 4/100
9/9 [==============================] - 18s 2s/step - loss: 3825896.0000
Epoch 5/100
9/9 [==============================] - 17s 2s/step - loss: 10103725.0000
Epoch 6/100
9/9 [==============================] - 35s 4s/step - loss: 10694811.0000
Epoch 7/100
9/9 [==============================] - 35s 4s/step - loss: 4396408.0000
Epoch 8/100
9/9 [==============================] - 35s 4s/step - loss: 2894632.5000
Epoch 9/100
9/9 [==============================] - 37s 4s/step - loss: 343333.6250
Epoch 10/100
9/9 [==============================] - 40s 4s/step - loss: 830264.2500
Epoch 11/100
9/9 [==============================] - 28s 3s/step - loss: 608722.7500
Epoch 12/100
9/9 [==============================] - 36s 4s/step - loss: 243491.6875
Epo

In [319]:
y_predict = np.array(mdl.predict(X_test)).reshape(-1)
calculate_mape(y_test, y_predict)

3/3 [==============================] - 0s 3ms/step


c:\Users\hamza\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


21.06985842878571

In [182]:
weight_matrix=mdl.layers[2].kernel
weight_matrix.shape

TensorShape([100, 89])

In [199]:
loss(weight_matrix, lengths, A, levels_left, levels_right)

C:\Users\hamza\AppData\Local\Temp\ipykernel_19716\3411038107.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.array(result)


NameError: name 'l' is not defined

In [193]:
reshape_weights(weight_matrix[0,:], lengths)

C:\Users\hamza\AppData\Local\Temp\ipykernel_19716\676212781.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.array(result)


array([<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-0.15434787], dtype=float32)>,
       <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 0.11945213, -0.08499871,  0.03614532,  0.1531064 ], dtype=float32)>,
       <tf.Tensor: shape=(28,), dtype=float32, numpy=
       array([ 0.10351485, -0.0306794 , -0.10432853, -0.13475315, -0.00930067,
               0.1767113 , -0.17145208, -0.06032456, -0.10896647, -0.12152006,
              -0.07783625, -0.1193021 ,  0.0134997 ,  0.04373387,  0.06839257,
              -0.1640523 ,  0.17149569, -0.15952419, -0.02312321,  0.0439762 ,
               0.15758695,  0.08671293,  0.12773558,  0.16239727,  0.1576011 ,
               0.08263822,  0.13436112,  0.11988743], dtype=float32)>         ,
       <tf.Tensor: shape=(56,), dtype=float32, numpy=
       array([-0.04328008, -0.05267015,  0.09754371,  0.1391737 ,  0.1596238 ,
               0.09861729, -0.0873127 , -0.05312444, -0.11267003,  0.08601849,
              -0.05830943, -0.00170921, 